In [1]:
##################
### BIBLIOTECAS ##
##################

import pandas as pd
import datetime as dt 
import time  
import tkinter as tk 
from tkinter import messagebox
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Alignment, Border, Side
import pygame
import random
import sys
from pathlib import Path
import os

pygame 2.6.1 (SDL 2.28.4, Python 3.12.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
class Base_Window:

    """
    Base class to set up window behaviors and shared methods.

    Attributes:
        Window (tk.Toplevel): The tkinter window instance.

    """

    def __init__(self, Title: str, Width: int = 1200, Height: int = 500) -> None:

        """
        Initializes a new window with the specified title, width, and height.

        Args:
            Title (str): The title of the window.
            Width (int): The width of the window. Defaults to 1200.
            Height (int): The height of the window. Defaults to 500.

        """

        self.Window = tk.Toplevel()
        self.Window.title(Title)
        self.Window.geometry(f"{Width}x{Height}")
        self.Center_Window(Width, Height)
        self.Lock_Window()

    def Center_Window(self, Width: int, Height: int) -> None:

        """
        Centers the window on the screen.

        Args:
            Width (int): The width of the window.
            Height (int): The height of the window.

        """
        Screen_Width = self.Window.winfo_screenwidth()
        Screen_Height = self.Window.winfo_screenheight()
        X_Position = (Screen_Width // 2) - (Width // 2)
        Y_Position = (Screen_Height // 2) - (Height // 2)
        self.Window.geometry(f"+{X_Position}+{Y_Position}")

    def Lock_Window(self) -> None:

        """
        Locks the window to prevent resizing or moving.

        """

        def Block_Movement(Event) -> None:

            """
            Callback to restrict the movement of the window.

            Args:
                Event: The tkinter event object.

            """

            self.Window.geometry(self.Window.geometry())

        self.Window.bind("<Configure>", Block_Movement)

    def Show(self) -> None:

        """
        Displays the window and manages its behavior.

        """

        self.Window.transient()  # Keep the window on top of its parent.
        self.Window.focus_force()  # Bring the window into focus.
        self.Window.grab_set()  # Prevent interaction with other windows.
        self.Window.protocol("WM_DELETE_WINDOW", self.Disable_Close)
        
    @staticmethod
    def Disable_Close() -> None:

        """
        Prevents the user from closing the window.
        
        """
        pass


In [ ]:
# Programa principal.
def Main():
    Window = tk.Tk() 
    Window.withdraw()  # Hide.
    Ventana = Base_Window('Titulo')
    Ventana.Show()
    Window.mainloop()

if __name__ == "__main__":
    Main()


In [ ]:
class Start_Window(Base_Window):

    """
    Window for starting a new task period.

    Attributes:
        App (Main_App): The main application instance.
        Plan_Entry (tk.Entry): Entry widget for the user's plan input.

    """

    def __init__(self, App: Main_App) -> None:

        """
        Initializes the start window for a new task period.

        Args:
            App (Main_App): The main application instance.

        """

        super().__init__("Nuevo período de trabajo")
        self.App = App
        self.Build_UI()

    def Build_UI(self) -> None:

        """
        Builds the UI components for the start window.

        """

        # Label asking the user what they plan to do in the next period.
        tk.Label(
            self.Window, 
            text=f"¿Qué vas a hacer en los próximos {self.App.Minutes_Period} minutos?", 
            font=("Calibri Light", 14)
        ).pack(pady=20)

        # Entry widget for the user to input their plan.
        self.Plan_Entry = tk.Entry(self.Window, font=("Calibri Light", 14))
        self.Plan_Entry.pack(pady=10)

        # Button to save the user's plan and start the timer.
        tk.Button(
            self.Window, 
            text="Empezar", 
            command=self.Save_Plan, 
            font=("Calibri Light", 14)
        ).pack(pady=20)

    def Save_Plan(self) -> None:

        """
        Saves the user's planned activity and starts the timer.

        The function adds a new row to the DataFrame with the plan, start time, and end time. It also saves the data.

        """

        Plan = self.Plan_Entry.get().strip()  # Retrieve the input plan.

        # Validate that the plan field is not empty.
        if not Plan:
            messagebox.showwarning("Campo vacío", "Por favor, ingresa un plan.")
            return

        # Create a new row for the planned activity.
        New_Row = {
            'INICIO': self.App.Start_Time,
            'FINAL': self.App.Start_Time + dt.timedelta(minutes=self.App.Minutes_Period),
            'PLAN_PREVISTO': Plan,
            'ACTIVIDAD_REALIZADA': '',
            'EXPLICACION': ''
        }

        # Append the new row to the DataFrame.
        self.App.Data_Frame = pd.concat([self.App.Data_Frame, pd.DataFrame([New_Row])], ignore_index=True)

        # Save the updated DataFrame to the file.
        self.App.Save_Data()

        # Close the current window.
        self.Window.destroy()

In [ ]:
class Activity_Window(Base_Window):

    """
    Window for reporting completed or uncompleted tasks.

    Attributes:
        App (Main_App): The main application instance.
        Activity_Entry (tk.Entry): Entry widget for activity input.
        Justify_Entry (tk.Entry): Entry widget for justification input.

    """

    def __init__(self, App: Main_App) -> None:

        """
        Initializes the activity reporting window.

        Args:
            App (Main_App): The main application instance.

        """

        super().__init__("Actividad realizada")
        self.App = App
        self.Build_UI()

    def Build_UI(self) -> None:

        """
        Builds the UI components for the window.

        """

        # Add label for activity prompt.
        tk.Label(
            self.Window, 
            text="¿Qué hiciste en lugar de lo previsto?", 
            font=("Calibri Light", 14)
        ).pack(pady=20)

        # Entry for entering activity.
        self.Activity_Entry = tk.Entry(self.Window, font=("Calibri Light", 14))
        self.Activity_Entry.pack(pady=10)

        # Add label for justification prompt.
        tk.Label(
            self.Window, 
            text="¿Por qué?", 
            font=("Calibri Light", 14)
        ).pack(pady=20)

        # Entry for entering justification.
        self.Justify_Entry = tk.Entry(self.Window, font=("Calibri Light", 14))
        self.Justify_Entry.pack(pady=10)

        # Button to save the activity and justification.
        tk.Button(
            self.Window, 
            text="Guardar", 
            command=self.Save_Activity, 
            font=("Calibri Light", 14)
        ).pack(pady=20)

    def Save_Activity(self) -> None:

        """
        Saves the reported activity and justification into the application's DataFrame.

        """
        
        Activity = self.Activity_Entry.get().strip()  # Retrieve the activity input.
        Justification = self.Justify_Entry.get().strip()  # Retrieve the justification input.

        # Validate that both fields are completed.
        if not Activity or not Justification:
            messagebox.showwarning("Campo vacío", "Por favor, completa todos los campos.")
            return

        # Update the application's DataFrame with the provided data.
        self.App.Data_Frame.loc[self.App.Last_Row_Index, 'ACTIVIDAD_REALIZADA'] = Activity
        self.App.Data_Frame.loc[self.App.Last_Row_Index, 'EXPLICACION'] = Justification

        # Save the updated DataFrame to the file.
        self.App.Save_Data()

        # Close the current window and open the Start Window.
        self.Window.destroy()
        Start_Window(self.App)

In [ ]:
class Main_App:

    """
    Main application to manage the flow and logic.

    Attributes:
        Data_Path (str): The file path to the Excel data file.
        Data_Frame (pd.DataFrame): The data loaded from the Excel file.
        Last_Row_Index (int): The index of the last row in the DataFrame.
        Minutes_Period (int): The interval period in minutes. Defaults to 25.
        Start_Time (datetime): The current start time.
        Phrase (str): The default motivational phrase.

    """

    def __init__(self, Data_Path: str) -> None:

        """
        Initializes the main application with data from the specified file path.

        Args:
            Data_Path (str): The file path to the Excel data file.

        """

        self.Data_Path = Data_Path
        self.Data_Frame = pd.read_excel(Data_Path)
        self.Last_Row_Index = len(self.Data_Frame) - 1 if len(self.Data_Frame) > 0 else 0
        self.Minutes_Period = 25
        self.Start_Time = dt.datetime.now()
        self.Phrase = "¡A trabajar!"

    def Start(self) -> None:

        """
        Determines which window to open initially based on the time difference.

        """

        if self.Time_Difference() <= 5 and len(self.Data_Frame) > 0:
            Activity_Window(self)
        else:
            Start_Window(self)

    def Time_Difference(self) -> int:

        """
        Calculates the time difference in minutes from the last activity.

        Returns:
            int: The difference in minutes from the last recorded activity.

        """

        if len(self.Data_Frame) > 0:
            Last_End_Time = self.Data_Frame.loc[self.Last_Row_Index, 'FINAL']
            if pd.notna(Last_End_Time):
                Last_End_Time = pd.to_datetime(Last_End_Time)
                return round((self.Start_Time - Last_End_Time).total_seconds() / 60)
        return 0

    def Save_Data(self) -> None:

        """
        Saves the current state of the DataFrame to the Excel file.
        
        """

        self.Data_Frame.to_excel(self.Data_Path, index=False)
        # Add column adjustments and formatting here if needed.